In [ ]:
#Import all the required dependencies
import os 
import sys
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Import Weather Dataset
weatherdata = os.path.abspath(os.path.join(os.getcwd(), '../Dataset/FirePredictor5DatasetNew.csv'))
WeatherData =  pd.read_csv(weatherdata)

In [ ]:
#Import Road Dataset
majorroad = os.path.abspath(os.path.join(os.getcwd(), '../Dataset/newroad.geojson'))
Road = geopandas.read_file(majorroad)

In [ ]:
#Import BC ZONE Dataset
BC = os.path.abspath(os.path.join(os.getcwd(), '../Dataset/BC_Dataset.csv'))
Zone = pd.read_csv(BC, low_memory = False)

In [ ]:
WeatherData.head()

In [6]:
Road.head()

,FEATURE_CODE,ENGLISH_NAME,FRENCH_NAME,OBJECTID,SE_ANNO_CAD_DATA,FEATURE_LENGTH_M,GEOMETRY.LEN,fme_feature_type,geometry
0,68,None,None,49,None,244120.5107,0,WHSE_BASEMAPPING.DBM_BC_7H_MIL_ROADS_LINE,"LINESTRING (-116.09422 58.33495, -116.04571 58..."
1,68,None,None,50,None,96955.8058,0,WHSE_BASEMAPPING.DBM_BC_7H_MIL_ROADS_LINE,"LINESTRING (-114.52986 56.41230, -114.55345 56..."
2,68,None,None,51,None,2409.9644,0,WHSE_BASEMAPPING.DBM_BC_7H_MIL_ROADS_LINE,"LINESTRING (-115.35796 56.48556, -115.35430 56..."
3,68,None,None,52,None,71613.7649,0,WHSE_BASEMAPPING.DBM_BC_7H_MIL_ROADS_LINE,"LINESTRING (-109.10190 55.99764, -109.09805 55..."
4,68,None,None,53,None,38921.1579,0,WHSE_BASEMAPPING.DBM_BC_7H_MIL_ROADS_LINE,"LINESTRING (-108.39704 55.77478, -108.38747 55..."


In [7]:
Zone.head()

,FIRE_NO,FIRE_YEAR,IGN_DATE,Ignition_Date,FIRE_CAUSE,FIRELABEL,FRCNTR,ZONE,FIRE_ID,FIRE_TYPE,GEO_DESC,LATITUDE,LONGITUDE,SIZE_HA,FCODE,SHAPE,OBJECTID,X,Y
0,R50182,2013,2.013080e+13,20130805,Person,2013-R50182,3.0,5.0,300182,Nuisance Fire,Upper Kitimat River Rec Site,54.2533,-128.4538,0.009,JA70003000,NaN,15266712,8.405403e+05,1.030646e+06
1,K61144,2009,2.009070e+13,20090730,Lightning,2009-K61144,5.0,6.0,501144,Fire,700m North Thali Lk,49.7768,-120.7232,0.009,JA70003000,NaN,15266713,1.379824e+06,5.431259e+05
2,K21111,2004,2.004050e+13,20040516,Person,2004-K21111,5.0,2.0,501111,Nuisance Fire,Duffy Lake,50.6557,-120.7091,NaN,JA70003000,NaN,15266714,1.373535e+06,6.407151e+05
3,K20312,2015,2.015070e+13,20150706,Lightning,2015-K20312,5.0,2.0,500312,Fire,Windy Lake,51.0901,-120.5062,2.400,JA70003000,NaN,15266715,1.384080e+06,6.900288e+05
4,N40516,2009,2.009070e+13,20090727,Lightning,2009-N40516,6.0,4.0,600516,Fire,Gold Stream river,51.7058,-118.3179,1.100,JA70003000,NaN,15266716,1.529123e+06,7.724545e+05


#### Select needed columns in the dataset

In [8]:
#Select the needed columns to merge 
ZoneData = Zone[['FIRE_YEAR','FIRE_CAUSE', 'FRCNTR', 'ZONE','LATITUDE', 'LONGITUDE','OBJECTID']]

In [9]:
#Function to round up our latitude and longtitude
def get_rounded_locations(df):
    df_lat_rounded = df.LATITUDE.map(lambda x: round(x * 4) / 4)
    df_lon_rounded = df.LONGITUDE.map(lambda x: round(x * 4) / 4)
    return df_lat_rounded, df_lon_rounded

In [10]:
df_lat_rounded, df_lon_rounded = get_rounded_locations(ZoneData)

In [11]:
#Rename the columns
ZoneData.rename(columns ={ 'LATITUDE': 'rounded_lat', 'LONGITUDE': 'rounded_lon' }, inplace=True )

/var/folders/1j/gsgy72qx2zg46_nk_jnw6_nm0000gn/T/ipykernel_25818/210403293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ZoneData.rename(columns ={ 'LATITUDE': 'rounded_lat', 'LONGITUDE': 'rounded_lon' }, inplace=True )


In [12]:
#Set Index to merge the dataset
ZoneData.set_index(['rounded_lat', 'rounded_lon'], inplace = True)

#### Create Geometry Point For Weather Data

In [13]:
WeatherData.head()

,UID_Fire,REF_ID,rounded_lon,rounded_lat,d2m_mean,d2m_std,t2m_mean,t2m_std,p140209_mean,p140209_std,...,T31_std,sample_mean,sample_std,FRP_mean,FRP_std,conf_mean,conf_std,type_mean,type_std,AoB_Category
0,629,BC-2015-V60407,-124.25,48.5,5.251866,1.583857,6.192056,1.389660,NaN,NaN,...,0.000000,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low
1,629,BC-2015-V60407,-124.25,48.5,5.251866,1.583857,6.192056,1.389660,NaN,NaN,...,3.606245,810.5,372.645274,75.7,97.863579,79.5,27.577164,0.0,0.0,Very Low
2,629,BC-2015-V60407,-124.25,48.5,6.255527,0.286837,6.756782,0.396061,NaN,NaN,...,0.000000,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low
3,629,BC-2015-V60407,-124.25,48.5,6.255527,0.286837,6.756782,0.396061,NaN,NaN,...,3.606245,810.5,372.645274,75.7,97.863579,79.5,27.577164,0.0,0.0,Very Low
4,629,BC-2015-V60407,-124.25,48.5,4.323322,1.206147,5.432946,0.680542,NaN,NaN,...,0.000000,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low


In [14]:
WeatherData.rename(columns ={ 'latitude': 'rounded_lat', 'longitude': 'rounded_lon' }, inplace=True )

In [15]:
geoweather = geopandas.GeoDataFrame(
    WeatherData, geometry=geopandas.points_from_xy(WeatherData.rounded_lon, WeatherData.rounded_lat))
geoweather.head()


,UID_Fire,REF_ID,rounded_lon,rounded_lat,d2m_mean,d2m_std,t2m_mean,t2m_std,p140209_mean,p140209_std,...,sample_mean,sample_std,FRP_mean,FRP_std,conf_mean,conf_std,type_mean,type_std,AoB_Category,geometry
0,629,BC-2015-V60407,-124.25,48.5,5.251866,1.583857,6.192056,1.389660,NaN,NaN,...,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low,POINT (-124.25000 48.50000)
1,629,BC-2015-V60407,-124.25,48.5,5.251866,1.583857,6.192056,1.389660,NaN,NaN,...,810.5,372.645274,75.7,97.863579,79.5,27.577164,0.0,0.0,Very Low,POINT (-124.25000 48.50000)
2,629,BC-2015-V60407,-124.25,48.5,6.255527,0.286837,6.756782,0.396061,NaN,NaN,...,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low,POINT (-124.25000 48.50000)
3,629,BC-2015-V60407,-124.25,48.5,6.255527,0.286837,6.756782,0.396061,NaN,NaN,...,810.5,372.645274,75.7,97.863579,79.5,27.577164,0.0,0.0,Very Low,POINT (-124.25000 48.50000)
4,629,BC-2015-V60407,-124.25,48.5,4.323322,1.206147,5.432946,0.680542,NaN,NaN,...,360.0,0.000000,138.8,0.000000,100.0,0.000000,0.0,0.0,Very Low,POINT (-124.25000 48.50000)


In [16]:
geoweather.set_index(['rounded_lon', 'rounded_lat'], inplace = True)

In [17]:
#Merge Weather Data And ZoneInfo Together
FireData = geoweather.merge(ZoneData, on=['rounded_lat', 'rounded_lon'], how='inner')

In [18]:
FireData.head()

UID_Fire          REF_ID  d2m_mean   d2m_std  \
rounded_lat rounded_lon                                                 
48.5        -123.5            283  BC-2016-V60440  5.535403  1.401351   
            -123.5            283  BC-2016-V60440  6.587591  0.691796   
            -123.5            283  BC-2016-V60440  4.810455  1.072378   
            -123.5            283  BC-2016-V60440  4.906936  0.305505   
            -123.5            283  BC-2016-V60440  5.355777  0.221331   

                         t2m_mean   t2m_std  p140209_mean  p140209_std  \
rounded_lat rounded_lon                                                  
48.5        -123.5       6.268164  1.586880           NaN          NaN   
            -123.5       7.625171  0.753571           NaN          NaN   
            -123.5       5.797542  0.963947           NaN          NaN   
            -123.5       5.822573  0.284743           NaN          NaN   
            -123.5       6.168260  0.317925           NaN          NaN   

                         cvh_mean  cvl_mean  ...  conf_std  type_mean  \
rounded_lat rounded_lon                      ...                        
48.5        -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   

                         type_std  AoB_Category                     geometry  \
rounded_lat rounded_lon                                                        
48.5        -123.5            0.0      Very Low  POINT (-123.50000 48.50000)   
            -123.5            0.0      Very Low  POINT (-123.50000 48.50000)   
            -123.5            0.0      Very Low  POINT (-123.50000 48.50000)   
            -123.5            0.0      Very Low  POINT (-123.50000 48.50000)   
            -123.5            0.0      Very Low  POINT (-123.50000 48.50000)   

                         FIRE_YEAR  FIRE_CAUSE  FRCNTR  ZONE  OBJECTID  
rounded_lat rounded_lon                                                 
48.5        -123.5            1994      Person     2.0   6.0  15441077  
            -123.5            1994      Person     2.0   6.0  15441077  
            -123.5            1994      Person     2.0   6.0  15441077  
            -123.5            1994      Person     2.0   6.0  15441077  
            -123.5            1994      Person     2.0   6.0  15441077  

[5 rows x 55 columns]

In [19]:
FireData.shape

(87420, 55)

#### Dealing With Road Dataset

In [20]:
#Only Needs the ID and geometry information to merge and calculate distance
roads = Road[['OBJECTID','geometry']]
roads.head()

,OBJECTID,geometry
0,49,"LINESTRING (-116.09422 58.33495, -116.04571 58..."
1,50,"LINESTRING (-114.52986 56.41230, -114.55345 56..."
2,51,"LINESTRING (-115.35796 56.48556, -115.35430 56..."
3,52,"LINESTRING (-109.10190 55.99764, -109.09805 55..."
4,53,"LINESTRING (-108.39704 55.77478, -108.38747 55..."


In [21]:
print('roads: ', roads.crs)
print('FireData: ', FireData.crs)

roads:  epsg:4326
FireData:  None


In [22]:
FireData.crs = "EPSG:4326"

##### Calculating the minimum distance between the points of wildfire and major roads 

In [23]:
# Calculate the minimum distance from fires to roads
def min_distance(point, lines):
    return lines.distance(point).min()

FireData['min_dist_to_roads'] = FireData['geometry'].apply(min_distance, args=(roads['geometry'],))
FireData.head()

/var/folders/1j/gsgy72qx2zg46_nk_jnw6_nm0000gn/T/ipykernel_25818/2781080187.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return lines.distance(point).min()


UID_Fire          REF_ID  d2m_mean   d2m_std  \
rounded_lat rounded_lon                                                 
48.5        -123.5            283  BC-2016-V60440  5.535403  1.401351   
            -123.5            283  BC-2016-V60440  6.587591  0.691796   
            -123.5            283  BC-2016-V60440  4.810455  1.072378   
            -123.5            283  BC-2016-V60440  4.906936  0.305505   
            -123.5            283  BC-2016-V60440  5.355777  0.221331   

                         t2m_mean   t2m_std  p140209_mean  p140209_std  \
rounded_lat rounded_lon                                                  
48.5        -123.5       6.268164  1.586880           NaN          NaN   
            -123.5       7.625171  0.753571           NaN          NaN   
            -123.5       5.797542  0.963947           NaN          NaN   
            -123.5       5.822573  0.284743           NaN          NaN   
            -123.5       6.168260  0.317925           NaN          NaN   

                         cvh_mean  cvl_mean  ...  type_mean  type_std  \
rounded_lat rounded_lon                      ...                        
48.5        -123.5       0.840935  0.057009  ...        0.0       0.0   
            -123.5       0.840935  0.057009  ...        0.0       0.0   
            -123.5       0.840935  0.057009  ...        0.0       0.0   
            -123.5       0.840935  0.057009  ...        0.0       0.0   
            -123.5       0.840935  0.057009  ...        0.0       0.0   

                         AoB_Category                     geometry  FIRE_YEAR  \
rounded_lat rounded_lon                                                         
48.5        -123.5           Very Low  POINT (-123.50000 48.50000)       1994   
            -123.5           Very Low  POINT (-123.50000 48.50000)       1994   
            -123.5           Very Low  POINT (-123.50000 48.50000)       1994   
            -123.5           Very Low  POINT (-123.50000 48.50000)       1994   
            -123.5           Very Low  POINT (-123.50000 48.50000)       1994   

                         FIRE_CAUSE  FRCNTR  ZONE  OBJECTID  min_dist_to_roads  
rounded_lat rounded_lon                                                         
48.5        -123.5           Person     2.0   6.0  15441077           0.013282  
            -123.5           Person     2.0   6.0  15441077           0.013282  
            -123.5           Person     2.0   6.0  15441077           0.013282  
            -123.5           Person     2.0   6.0  15441077           0.013282  
            -123.5           Person     2.0   6.0  15441077           0.013282  

[5 rows x 56 columns]

In [26]:
FireData['min_dist_to_roads'].unique()

array([0.01328225, 0.03110291, 0.22535841, 0.09790902, 0.09592111,
       0.56522154, 0.01506893, 0.06460564, 0.00381494, 0.18200755,
       0.02724399, 0.14614615, 0.00910513, 0.15051402, 0.17583045,
       0.23602287, 0.01152713, 0.02636488, 0.27828776, 0.04893999,
       0.1484726 , 0.00264791, 0.04331679, 0.17247268, 0.01917537,
       0.0991642 , 0.05041874, 0.25070762, 0.76158063, 0.01666718,
       0.03102531, 0.12514425, 0.01006474, 0.01518867, 0.04596823,
       0.32003875, 0.09561794, 0.00926268, 1.49546728])

In [27]:
#Import Fire Center Location
FireCenter = os.path.abspath(os.path.join(os.getcwd(), '../Dataset/FireCenterLocation.geojson'))
FireCenter = geopandas.read_file(FireCenter)

In [28]:
FireCenter.head()

,MOF_FIRE_CENTRE_ID,MOF_FIRE_CENTRE_NAME,OBJECTID,SE_ANNO_CAD_DATA,FEATURE_AREA_SQM,FEATURE_LENGTH_M,GEOMETRY.AREA,GEOMETRY.LEN,fme_feature_type,geometry
0,161,Southeast Fire Centre,1,None,8.233825e+10,1.952981e+06,0,0,WHSE_LEGAL_ADMIN_BOUNDARIES.DRP_MOF_FIRE_CENTR...,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."
1,162,Kamloops Fire Centre,2,None,7.157919e+10,2.517696e+06,0,0,WHSE_LEGAL_ADMIN_BOUNDARIES.DRP_MOF_FIRE_CENTR...,"POLYGON ((-120.13900 52.87591, -120.13914 52.8..."
2,163,Cariboo Fire Centre,3,None,8.248007e+10,2.088550e+06,0,0,WHSE_LEGAL_ADMIN_BOUNDARIES.DRP_MOF_FIRE_CENTR...,"POLYGON ((-122.00253 53.42121, -122.00251 53.4..."
3,164,Coastal Fire Centre,4,None,2.034292e+11,3.758998e+06,0,0,WHSE_LEGAL_ADMIN_BOUNDARIES.DRP_MOF_FIRE_CENTR...,"POLYGON ((-131.69266 54.69807, -132.69892 54.6..."
4,165,Prince George Fire Centre,5,None,3.367330e+11,4.919934e+06,0,0,WHSE_LEGAL_ADMIN_BOUNDARIES.DRP_MOF_FIRE_CENTR...,"POLYGON ((-120.00135 59.99000, -120.00134 60.0..."


In [29]:
FireCenter = FireCenter[['geometry' ,'OBJECTID' ]]

In [30]:
#Create New Object ID and drop the old object id to merge
FireData = FireData.drop(['OBJECTID', 'geometry'],  axis = 1)

###### Create new condition for object id 


In [31]:
conditions=[
    (FireData['FRCNTR']==2.0),
    (FireData['FRCNTR']==3.0), 
    (FireData['FRCNTR']==4.0),
    (FireData['FRCNTR']==5.0),
    (FireData['FRCNTR']==6.0), 
    (FireData['FRCNTR']==7.0),
]
choices = [1, 2, 3, 4, 5, 6]

FireData['OBJECTID'] = np.select(conditions, choices, default = 1)

In [32]:
FireData.head()

UID_Fire          REF_ID  d2m_mean   d2m_std  \
rounded_lat rounded_lon                                                 
48.5        -123.5            283  BC-2016-V60440  5.535403  1.401351   
            -123.5            283  BC-2016-V60440  6.587591  0.691796   
            -123.5            283  BC-2016-V60440  4.810455  1.072378   
            -123.5            283  BC-2016-V60440  4.906936  0.305505   
            -123.5            283  BC-2016-V60440  5.355777  0.221331   

                         t2m_mean   t2m_std  p140209_mean  p140209_std  \
rounded_lat rounded_lon                                                  
48.5        -123.5       6.268164  1.586880           NaN          NaN   
            -123.5       7.625171  0.753571           NaN          NaN   
            -123.5       5.797542  0.963947           NaN          NaN   
            -123.5       5.822573  0.284743           NaN          NaN   
            -123.5       6.168260  0.317925           NaN          NaN   

                         cvh_mean  cvl_mean  ...  conf_std  type_mean  \
rounded_lat rounded_lon                      ...                        
48.5        -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   
            -123.5       0.840935  0.057009  ...       0.0        0.0   

                         type_std  AoB_Category  FIRE_YEAR  FIRE_CAUSE  \
rounded_lat rounded_lon                                                  
48.5        -123.5            0.0      Very Low       1994      Person   
            -123.5            0.0      Very Low       1994      Person   
            -123.5            0.0      Very Low       1994      Person   
            -123.5            0.0      Very Low       1994      Person   
            -123.5            0.0      Very Low       1994      Person   

                         FRCNTR  ZONE  min_dist_to_roads  OBJECTID  
rounded_lat rounded_lon                                             
48.5        -123.5          2.0   6.0           0.013282         1  
            -123.5          2.0   6.0           0.013282         1  
            -123.5          2.0   6.0           0.013282         1  
            -123.5          2.0   6.0           0.013282         1  
            -123.5          2.0   6.0           0.013282         1  

[5 rows x 55 columns]

In [33]:
FireCenter.reset_index()

,index,geometry,OBJECTID
0,0,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",1
1,1,"POLYGON ((-120.13900 52.87591, -120.13914 52.8...",2
2,2,"POLYGON ((-122.00253 53.42121, -122.00251 53.4...",3
3,3,"POLYGON ((-131.69266 54.69807, -132.69892 54.6...",4
4,4,"POLYGON ((-120.00135 59.99000, -120.00134 60.0...",5
5,5,"POLYGON ((-138.00034 60.00092, -139.06131 59.9...",6


In [34]:
FireCenter.columns

Index(['geometry', 'OBJECTID'], dtype='object')

In [35]:
FireData.columns

Index(['UID_Fire', 'REF_ID', 'd2m_mean', 'd2m_std', 't2m_mean', 't2m_std',
       'p140209_mean', 'p140209_std', 'cvh_mean', 'cvl_mean', 'ptype_mean',
       'ptype_std', 'sst_mean', 'sst_std', 'sf_mean', 'sf_std', 'slt_mean',
       'slt_std', 'sp_mean', 'sp_std', 'tp_mean', 'tp_std', 'tvh_mean',
       'tvh_std', 'wind_speed_mean', 'wind_speed_std', 'date', 'Total_AoB',
       'Status', 'sat', 'HHMM_mean', 'HHMM_std', 'lat_mean', 'lat_std',
       'lon_mean', 'lon_std', 'T21_mean', 'T21_std', 'T31_mean', 'T31_std',
       'sample_mean', 'sample_std', 'FRP_mean', 'FRP_std', 'conf_mean',
       'conf_std', 'type_mean', 'type_std', 'AoB_Category', 'FIRE_YEAR',
       'FIRE_CAUSE', 'FRCNTR', 'ZONE', 'min_dist_to_roads', 'OBJECTID'],
      dtype='object')

In [36]:
FireDataset = FireData.merge(FireCenter, on=['OBJECTID'], how='inner')

In [37]:
FireDataset.head()

,UID_Fire,REF_ID,d2m_mean,d2m_std,t2m_mean,t2m_std,p140209_mean,p140209_std,cvh_mean,cvl_mean,...,type_mean,type_std,AoB_Category,FIRE_YEAR,FIRE_CAUSE,FRCNTR,ZONE,min_dist_to_roads,OBJECTID,geometry
0,283,BC-2016-V60440,5.535403,1.401351,6.268164,1.586880,NaN,NaN,0.840935,0.057009,...,0.0,0.0,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."
1,283,BC-2016-V60440,6.587591,0.691796,7.625171,0.753571,NaN,NaN,0.840935,0.057009,...,0.0,0.0,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."
2,283,BC-2016-V60440,4.810455,1.072378,5.797542,0.963947,NaN,NaN,0.840935,0.057009,...,0.0,0.0,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."
3,283,BC-2016-V60440,4.906936,0.305505,5.822573,0.284743,NaN,NaN,0.840935,0.057009,...,0.0,0.0,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."
4,283,BC-2016-V60440,5.355777,0.221331,6.168260,0.317925,NaN,NaN,0.840935,0.057009,...,0.0,0.0,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4..."


In [40]:
FireDataset = geopandas.GeoDataFrame(FireDataset, crs=4326)
FireDataset = FireDataset.to_crs(4326)

In [41]:
#Create a lon and lat to calculate distance
FireDataset['lon'] = FireDataset.centroid.x  
FireDataset['lat'] = FireDataset.centroid.y

/var/folders/1j/gsgy72qx2zg46_nk_jnw6_nm0000gn/T/ipykernel_25818/3116153945.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  FireDataset['lon'] = FireDataset.centroid.x
/var/folders/1j/gsgy72qx2zg46_nk_jnw6_nm0000gn/T/ipykernel_25818/3116153945.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  FireDataset['lat'] = FireDataset.centroid.y


In [42]:
  FireDataset.head()

,UID_Fire,REF_ID,d2m_mean,d2m_std,t2m_mean,t2m_std,p140209_mean,p140209_std,cvh_mean,cvl_mean,...,AoB_Category,FIRE_YEAR,FIRE_CAUSE,FRCNTR,ZONE,min_dist_to_roads,OBJECTID,geometry,lon,lat
0,283,BC-2016-V60440,5.535403,1.401351,6.268164,1.586880,NaN,NaN,0.840935,0.057009,...,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",-116.962727,50.320102
1,283,BC-2016-V60440,6.587591,0.691796,7.625171,0.753571,NaN,NaN,0.840935,0.057009,...,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",-116.962727,50.320102
2,283,BC-2016-V60440,4.810455,1.072378,5.797542,0.963947,NaN,NaN,0.840935,0.057009,...,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",-116.962727,50.320102
3,283,BC-2016-V60440,4.906936,0.305505,5.822573,0.284743,NaN,NaN,0.840935,0.057009,...,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",-116.962727,50.320102
4,283,BC-2016-V60440,5.355777,0.221331,6.168260,0.317925,NaN,NaN,0.840935,0.057009,...,Very Low,1994,Person,2.0,6.0,0.013282,1,"POLYGON ((-117.99119 52.49707, -117.99129 52.4...",-116.962727,50.320102


In [44]:
FireDataset.columns

Index(['UID_Fire', 'REF_ID', 'd2m_mean', 'd2m_std', 't2m_mean', 't2m_std',
       'p140209_mean', 'p140209_std', 'cvh_mean', 'cvl_mean', 'ptype_mean',
       'ptype_std', 'sst_mean', 'sst_std', 'sf_mean', 'sf_std', 'slt_mean',
       'slt_std', 'sp_mean', 'sp_std', 'tp_mean', 'tp_std', 'tvh_mean',
       'tvh_std', 'wind_speed_mean', 'wind_speed_std', 'date', 'Total_AoB',
       'Status', 'sat', 'HHMM_mean', 'HHMM_std', 'lat_mean', 'lat_std',
       'lon_mean', 'lon_std', 'T21_mean', 'T21_std', 'T31_mean', 'T31_std',
       'sample_mean', 'sample_std', 'FRP_mean', 'FRP_std', 'conf_mean',
       'conf_std', 'type_mean', 'type_std', 'AoB_Category', 'FIRE_YEAR',
       'FIRE_CAUSE', 'FRCNTR', 'ZONE', 'min_dist_to_roads', 'OBJECTID',
       'geometry', 'lon', 'lat'],
      dtype='object')

In [47]:
import geopy.distance

lon1 =FireDataset['lon_mean']
lon2 = FireDataset['lon']
lat1 = FireDataset['lat_mean']
lat2 = FireDataset['lat']


def distance(lat1,lon1,  lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geopy.distance.geodesic(coords_1, coords_2).km

FireDataset['distanceinkm'] = [distance(FireDataset.lat_mean[i], FireDataset.lon_mean[i],FireDataset.lat[i], FireDataset.lon[i]) for i in range(len(FireDataset))]


In [48]:
#Get the fire centers and fire 
FireDataset.drop(['p140209_mean','p140209_std'], axis = 1, inplace = True)

In [50]:
FireDataset.drop(['UID_Fire','REF_ID', 'OBJECTID', 'geometry', 'FIRE_YEAR', 'ZONE'], axis = 1, inplace = True)

In [51]:
FireDataset.head()

,d2m_mean,d2m_std,t2m_mean,t2m_std,cvh_mean,cvl_mean,ptype_mean,ptype_std,sst_mean,sst_std,...,conf_std,type_mean,type_std,AoB_Category,FIRE_CAUSE,FRCNTR,min_dist_to_roads,lon,lat,distanceinkm
0,5.535403,1.401351,6.268164,1.586880,0.840935,0.057009,1.000046,0.000000,NaN,NaN,...,0.0,0.0,0.0,Very Low,Person,2.0,0.013282,-116.962727,50.320102,513.680113
1,6.587591,0.691796,7.625171,0.753571,0.840935,0.057009,1.000046,0.000000,NaN,NaN,...,0.0,0.0,0.0,Very Low,Person,2.0,0.013282,-116.962727,50.320102,513.680113
2,4.810455,1.072378,5.797542,0.963947,0.840935,0.057009,0.875040,0.337847,NaN,NaN,...,0.0,0.0,0.0,Very Low,Person,2.0,0.013282,-116.962727,50.320102,513.680113
3,4.906936,0.305505,5.822573,0.284743,0.840935,0.057009,1.000046,0.000000,NaN,NaN,...,0.0,0.0,0.0,Very Low,Person,2.0,0.013282,-116.962727,50.320102,513.680113
4,5.355777,0.221331,6.168260,0.317925,0.840935,0.057009,0.916709,0.282343,NaN,NaN,...,0.0,0.0,0.0,Very Low,Person,2.0,0.013282,-116.962727,50.320102,513.680113


In [52]:
FireDataset.to_csv('FireFinalDataset.csv')

In [ ]:
import pandas as pd
import os
from mapboxgl.utils import *
from mapboxgl.viz import *

# Must be a public token, starting with `pk`

# Create a geojson Feature Collection export from a Pandas dataframe
points = df_to_geojson(weatherdata, 
                       properties=['date', 'FRCNTR', 'FIRE_CAUSE', 'AoB_Category'],
                       lat='lat_mean', lon='lon_mean', precision=3)